In [ ]:
import os
os.chdir("../")

In [ ]:
!pip install scipy

In [ ]:
from dataclasses import dataclass
from pathlib import Path

#instead of using namedtuple approach, we can use below approach of class variables
#advantage of this approach is here we can define datatype of each variable.

###################################

@dataclass(frozen=True)
class PrepareCallbacksConfig:
  root_dir: Path
  tensorboard_root_log_dir: Path
  checkpoint_model_filepath: Path

####################################

@dataclass(frozen=True)
class TrainingConfig:
  root_dir: Path
  trained_model_path: Path
  updated_base_model_path: Path
  training_data: Path
  params_epochs: int
  params_batch_size: int
  params_is_augmentation: bool
  params_image_size: list

  

In [ ]:
from deepClassifier.constants import *
from deepClassifier.utils import read_yaml, create_directories
import tensorflow as tf

In [ ]:
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath) #reading config file
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    ###################################

    def get_prepare_callback_config(self) -> PrepareCallbacksConfig: #o/p datatype will be PrepareCallbacksConfig
        config = self.config.prepare_callbacks #passing data to local config variable. This data prepare_callbacks comes from config.yaml file
        
        #creating directory not only for root but for checkpoint and tensorboard logs.
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath) #dirname gives dir filename/filepath for model checkpoint. This data comes from config.yaml file
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ]) 

        prepare_callbacks_config = PrepareCallbacksConfig( #using class tuple to get o/p datatype as PrepareCallbacksConfig
            root_dir = Path(config.root_dir),
            tensorboard_root_log_dir = Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath = Path(config.checkpoint_model_filepath)
        )
        return prepare_callbacks_config

    ###################################

    def get_training_config(self) -> TrainingConfig: #o/p datatype will be TrainingConfig
        training = self.config.training #passing data to local config variable. This data training comes from config.yaml file
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir,"PetImages")
        #creating directory for root
        create_directories([Path(training.root_dir)])

        training_config = TrainingConfig( #using class tuple to get o/p datatype as TrainingConfig
            root_dir = Path(training.root_dir),
            trained_model_path = Path(training.trained_model_path),
            updated_base_model_path = Path(prepare_base_model.updated_base_model_path), 
            training_data = Path(training_data),
            params_epochs = params.EPOCHS,
            params_batch_size = params.BATCH_SIZE,
            params_is_augmentation = params.AUGMENTATION,
            params_image_size = params.IMAGE_SIZE
        )
        return training_config

In [ ]:
import os
import urllib.request as request #to work on url (while downloading)
from zipfile import ZipFile # for zip
import tensorflow as tf
import time

#creating callback component
#Callback is a method where we save the model from time to time so that in case our training is stopped by some reason, then we can start with latest model.
#It also maintains tensorflow logs so we get to know the status of training, accuracy and loss status, etc
#Callback examples are checkpoint callbacks, early stopping callbacks, tensorboard logs callbacks, etc.
#Checkpoint callback is similar to autosave after certain iterations/time steps in CFD workflow, whereas early stopping is similar to convergence achieved within certain threshold, and, tensorboard logs is like logfiles of .sim

class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig): #takes input in local config variable with i/p datatype as PrepareCallbacksConfig
        self.config = config

    # single _ is used as a nomenclature for hidden method.
    @property #property decorator
    def _create_tb_callbacks(self):
        #creating directories with timestamp
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S") #timestamp use for creating unique directories. dir name will be Year-month-date-hr-min-sec
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}"
            )        
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True #save only best weights
        )

    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks, #for this fun we are not using (), bcoz we have decorated this function with @property decorator.
            self._create_ckpt_callbacks
        ]
    

In [ ]:
import os
import urllib.request as request #to work on url (while downloading)
from zipfile import ZipFile # for zip
import tensorflow as tf
import time

class Training:
    def __init__(self, config: TrainingConfig): #takes input in local config variable with i/p datatype as TrainingConfig
        self.config = config

    #loading the base model
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )
        
    # below function is used for augmentation. i.e it generates training and validation data if required as per conditions given to it.
    # augmentation is done when we have small dataset and we need more data for training purpose.
    # in augumentation, dataset of new images is created by rotating, fliping, streaching, zooming, etc an existing image.
    # hence it much more data, just like random seed. 
    def train_valid_generator(self):

        datagenerator_kwargs = dict( #it is a dictionary
            rescale = 1./255,
            validation_split = 0.20
        )

        dataflow_kwargs = dict( #it is a dictionary
            target_size = self.config.params_image_size [:-1], #channel info of image is not required, hence we take -1, means all rows and coloumns except last.
            batch_size = self.config.params_batch_size,
            interpolation = "bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs #passing the dict as kwargs to this function. This helps to pass multiple args at once
        )

        # creating validation data generator
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        # we don't need any augmentation for validation data as it is raw data that we want.
        # but for training purpose we need it.
        if self.config.params_is_augmentation: #execute only if augmentation is set to True
            # below function generates data from an image (just like random seed). These can also be kept in parameters file and controlled from there.
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range= 40, #rotate image by 40 radians.
                horizontal_flip=True, #horizantally flipping
                width_shift_range=0.2, # streach image by 20% horizontally by interpolation, means zoom in by 20%
                height_shift_range=0.2,# streach image by 20% vertically by interpolation
                shear_range=0.2, #like shear force, tilt image by 20%
                zoom_range=0.2, #zoom in/out
                **datagenerator_kwargs #also apply params from these dict
            )
        else: #keep the datagenerator same
            training_datagenerator = valid_datagenerator

         # creating training data generator
        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model): #this is static method hence doesn't need self.
        model.save(path)

    def train(self, callback_list: list):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size # no of steps per epoch = sample size/batch size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs = self.config.params_epochs,
            steps_per_epoch = self.steps_per_epoch,
            validation_steps = self.validation_steps,
            validation_data = self.valid_generator,
            callbacks = callback_list
        )

        self.save_model(
            path = self.config.trained_model_path,
            model = self.model
        )




In [ ]:
#pipeline section
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config = prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()


    training_config = config.get_training_config()
    training = Training(config = training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train(
        callback_list = callback_list
    )
except Exception as e:
    raise e

In [ ]:
def example(x,**kwargs):
    print(locals())

In [ ]:
example(x=3)

In [ ]:
example(x=3, y=4,z=6)

In [ ]:
extra = dict(y=4, z=55)
example(x=3, **extra) #here giving extra as kwargs helps us to reuse it. we can add/sub multiple variables in extra and then put it in example function